# **Import Modules**

In [40]:
!pip install optuna imbalanced-learn gensim -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [2]:
!pip install scipy==1.10.1 gensim==4.3.1 numpy==1.24.3 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 38.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cvxpy 1.6.5 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
scikit-image 0.25.2 requires scipy>=1.11.4, but you have scipy 1.10.1 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
albumentations 2.0.6 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.

In [13]:
import pandas as pd
import numpy as np
import optuna
import re
import string
import warnings

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.base import BaseEstimator, TransformerMixin
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# **Load Data**

In [2]:
!wget --no-check-certificate https://raw.githubusercontent.com/rioooranteai/nlp-project/main/Analisis%20Sentimen%20-%20Coretax/Dataset/Prep_Version-2/With-Lemma/Dataset_roberta.csv -O "/content/Dataset.csv"

--2025-05-06 12:31:29--  https://raw.githubusercontent.com/rioooranteai/nlp-project/main/Analisis%20Sentimen%20-%20Coretax/Dataset/Prep_Version-2/With-Lemma/Dataset_roberta.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3220631 (3.1M) [text/plain]
Saving to: ‘/content/Dataset.csv’

/content/Dataset.cs 100%[===================>]   3.07M  --.-KB/s    in 0.08s   

2025-05-06 12:31:29 (39.3 MB/s) - ‘/content/Dataset.csv’ saved [3220631/3220631]



In [3]:
df = pd.read_csv("/content/Dataset.csv", usecols=['full_text', 'sentiment'])
df.head()

,full_text,sentiment
0,dizol97 hai kak . mohon maaf atas tidaknyama n...,neutral
1,dianariyanirs hai kak . mohon maaf atas tidakn...,neutral
2,songofdaenerys hai kak . alamat kakak pindah w...,neutral
3,hhaaai! sha open jasa ges lapor spt masa pph 2...,neutral
4,kring _pajak ditjenpajakri pajak bayar lewat s...,neutral


In [4]:
df.tail()

,full_text,sentiment
15365,pns_ababil gw curiga server sakti dipake buat ...,negative
15366,kring _pajak bihunbaso kak hari kringpajak cob...,neutral
15367,lunarztar kayak pernah denger coretax deh. . ....,negative
15368,setuju sangat setuju! individu bayar pajak san...,negative
15369,c_adburry sering dapet kinini data kak coretax,neutral


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15370 entries, 0 to 15369
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   full_text  15370 non-null  object
 1   sentiment  15370 non-null  object
dtypes: object(2)
memory usage: 240.3+ KB


# **Preprocessing Data**

In [6]:
def preprocessing(teks):
    teks = teks.translate(str.maketrans('', '', string.punctuation))

    teks = re.sub(r'\s+', ' ', teks)

    return teks.strip()

df['full_text'] = df['full_text'].apply(preprocessing)

# **Split Data**

In [7]:
X = df['full_text']
y = df['sentiment']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# **Feature Extraction, Handling Imbalance Data, Training Model, Evaluation, & Inference**

## **TF-IDF**

### Model Selection

In [ ]:
# Tanpa Handling Imbalance Data
models_tfidf = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Support Vector Machine': SVC(),
    'Naive Bayes': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models_tfidf.items():
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        (model_name, model)
    ])

    scores = cross_val_score(pipeline, X, y, cv=5)
    print(f'{model_name} Accuracy: {scores.mean():.4f}')

Logistic Regression Accuracy: 0.8349
Random Forest Accuracy: 0.8118
Gradient Boosting Accuracy: 0.7975
Support Vector Machine Accuracy: 0.8312
Naive Bayes Accuracy: 0.8041
Decision Tree Accuracy: 0.7405


In [ ]:
# Dengan Handling Imbalance Data
models_tfidf = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Support Vector Machine': SVC(),
    'Naive Bayes': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models_tfidf.items():
    pipeline = ImbPipeline([
        ('tfidf', TfidfVectorizer()),
        ('smote', SMOTE(random_state=42)),
        ('clf', model)
    ])

    scores = cross_val_score(pipeline, X, y, cv=5)
    print(f'{model_name} Accuracy: {scores.mean():.4f}')

Logistic Regression Accuracy: 0.8220
Random Forest Accuracy: 0.8116
Gradient Boosting Accuracy: 0.7522
Support Vector Machine Accuracy: 0.8324
Naive Bayes Accuracy: 0.7999
Decision Tree Accuracy: 0.7254


Berdasarkan proses model selection, maka model yang akan digunakan adalah Logistic tanpa handling imbalance Data dan SVM dengan Handling Imbalance

### **Logistic Regression**

#### Hyperparamter Tuning Best Model

In [ ]:
def objective(trial):
    tfidf_params = {
        "max_df": trial.suggest_float("max_df", 0.5, 1.0),
        "min_df": trial.suggest_int("min_df", 1, 10),
        "ngram_range": trial.suggest_categorical("ngram_range", [(1, 1), (1, 2)]),
        "use_idf": trial.suggest_categorical("use_idf", [True, False]),
        "sublinear_tf": trial.suggest_categorical("sublinear_tf", [True, False]),
    }

    lr_params = {
        "C": trial.suggest_loguniform("C", 1e-4, 100),
        "solver": trial.suggest_categorical("solver", ["liblinear", "lbfgs"]),
        "max_iter": trial.suggest_int("max_iter", 100, 1000),
        "random_state": 42,
    }

    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(**tfidf_params)),
        ('clf', LogisticRegression(**lr_params))
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    return accuracy_score(y_test, y_pred)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2025-05-04 14:03:10,041] A new study created in memory with name: no-name-dbf98299-9a1b-48af-a57a-74b5d5937325
[I 2025-05-04 14:03:12,540] Trial 0 finished with value: 0.5605074821080026 and parameters: {'max_df': 0.9560263590117324, 'min_df': 1, 'ngram_range': (1, 2), 'use_idf': True, 'sublinear_tf': False, 'C': 0.00021251985997107123, 'solver': 'liblinear', 'max_iter': 190}. Best is trial 0 with value: 0.5605074821080026.
[I 2025-05-04 14:03:20,132] Trial 1 finished with value: 0.8214053350683149 and parameters: {'max_df': 0.5597544395171952, 'min_df': 3, 'ngram_range': (1, 1), 'use_idf': False, 'sublinear_tf': True, 'C': 39.489492485428016, 'solver': 'lbfgs', 'max_iter': 303}. Best is trial 1 with value: 0.8214053350683149.
[I 2025-05-04 14:03:23,303] Trial 2 finished with value: 0.8405985686402082 and parameters: {'max_df': 0.8855450477667097, 'min_df': 9, 'ngram_range': (1, 2), 'use_idf': False, 'sublinear_tf': False, 'C': 1.6033038796828354, 'solver': 'lbfgs', 'max_iter': 270}

#### Train Model

In [ ]:
print("Best Parameters:")
for k, v in study.best_params.items():
    print(f"  {k}: {v}")

Best Parameters:
  max_df: 0.9508799995508697
  min_df: 6
  ngram_range: (1, 2)
  use_idf: False
  sublinear_tf: True
  C: 1.304392892275882
  solver: lbfgs
  max_iter: 685


In [ ]:
pipeline_tfidf_lr = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_df=0.9508799995508697,
        min_df=6,
        ngram_range=(1, 2),
        use_idf=False,
        sublinear_tf=True
    )),
    ('clf', LogisticRegression(
        C=1.304392892275882,
        solver='lbfgs',
        max_iter=685,
        random_state=42
    ))
])

pipeline_tfidf_lr.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.9508799995508697, min_df=6,
                                 ngram_range=(1, 2), sublinear_tf=True,
                                 use_idf=False)),
                ('clf',
                 LogisticRegression(C=1.304392892275882, max_iter=685,
                                    random_state=42))])

#### Evaluation

In [ ]:
y_pred = pipeline_tfidf_lr.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.79      0.88      0.84      1189
     neutral       0.90      0.90      0.90      1723
    positive       0.65      0.14      0.22       162

    accuracy                           0.85      3074
   macro avg       0.78      0.64      0.65      3074
weighted avg       0.85      0.85      0.84      3074



#### Inference

In [ ]:
print(f"Text -> {X_test.iloc[1]}")
print(f"Label -> {y_test.iloc[1]}")
print(f"Prediction -> {pipeline_tfidf_lr.predict([X_test.iloc[1]])[0]}")

Text -> min mau nanya jadi kantor beli barang bulan maret pm sebut ditit aja waktu lapor masa februari min kredit waktu lapor ppn maret aja min kring pajak
Label -> neutral
Prediction -> neutral


### **Support Vector Machine**

#### Hyperparameter Tuning

In [ ]:
def objective_svm(trial):
    svm_params = {
        "C": trial.suggest_loguniform("C", 1e-4, 100),
        "kernel": trial.suggest_categorical("kernel", ["linear", "rbf", "poly", "sigmoid"]),
        "gamma": trial.suggest_categorical("gamma", ["scale", "auto"]),
        "probability": True,
        "class_weight": trial.suggest_categorical("class_weight", ["balanced", None]),
        "random_state": 42,
    }

    if svm_params["kernel"] == "poly":
        svm_params["degree"] = trial.suggest_int("degree", 2, 5)

    tfidf_params = {
        "max_features": trial.suggest_categorical("max_features", [None, 1000, 5000, 10000]),
        "min_df": trial.suggest_categorical("min_df", [1, 2, 5]),
        "max_df": trial.suggest_float("max_df", 0.5, 1.0),
        "ngram_range": trial.suggest_categorical(
            "ngram_range",
            [(1, 1), (1, 2), (1, 3)]
        ),
    }

    smote_params = {
        "k_neighbors": trial.suggest_int("k_neighbors", 3, 10),
        "random_state": 42
    }

    pipeline = ImbPipeline([
        ('tfidf', TfidfVectorizer(**tfidf_params)),
        ('smote', SMOTE(**smote_params)),
        ('clf', SVC(**svm_params))
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    return accuracy_score(y_test, y_pred)

study_svm = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5)
)

study_svm.optimize(
    objective_svm,
    n_trials=200,
    n_jobs=-1,
    timeout=3 * 60 * 60
);

[I 2025-05-05 04:12:32,566] A new study created in memory with name: no-name-d3acb282-8be1-4339-9ce5-73044ce57acc
[I 2025-05-05 04:43:55,555] Trial 16 finished with value: 0.6089794348206035 and parameters: {'C': 3.773156057898387, 'kernel': 'sigmoid', 'gamma': 'scale', 'class_weight': 'balanced', 'max_features': 1000, 'min_df': 5, 'max_df': 0.9979192216433432, 'ngram_range': (1, 1), 'k_neighbors': 7}. Best is trial 16 with value: 0.6089794348206035.
[I 2025-05-05 04:53:36,858] Trial 8 finished with value: 0.6770479571932007 and parameters: {'C': 22.263982642856718, 'kernel': 'linear', 'gamma': 'auto', 'class_weight': None, 'max_features': None, 'min_df': 2, 'max_df': 0.5203897669946611, 'ngram_range': (1, 1), 'k_neighbors': 10}. Best is trial 8 with value: 0.6770479571932007.
[I 2025-05-05 05:11:11,654] Trial 2 finished with value: 0.6653450925234418 and parameters: {'C': 8.018034186621078, 'kernel': 'linear', 'gamma': 'scale', 'class_weight': None, 'max_features': 1000, 'min_df': 2, 

#### Train Model
Hasil dari hyperparameter tuning ternyata tidak lebih baik dari default, sehingga untuk train tetap menggunakan parameter default.

In [14]:
pipeline_tfidf_svm = ImbPipeline([
    ('tfidf', TfidfVectorizer()),
    ('smote', SMOTE(random_state=42)),
    ('clf', SVC()),
])

pipeline_tfidf_svm.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('smote', SMOTE(random_state=42)),
                ('clf', SVC())])

#### Evaluation

In [16]:
y_pred_SVM = pipeline_tfidf_svm.predict(X_test)

print(classification_report(y_test, y_pred_SVM))

              precision    recall  f1-score   support

    negative       0.78      0.89      0.83      1189
     neutral       0.91      0.88      0.90      1723
    positive       0.54      0.22      0.31       162

    accuracy                           0.85      3074
   macro avg       0.74      0.66      0.68      3074
weighted avg       0.84      0.85      0.84      3074



#### Inference

In [17]:
print(f"Text -> {X_test.iloc[1]}")
print(f"Label -> {y_test.iloc[1]}")
print(f"Prediction -> {pipeline_tfidf_svm.predict([X_test.iloc[1]])[0]}")

Text -> min mau nanya jadi kantor beli barang bulan maret pm sebut ditit aja waktu lapor masa februari min kredit waktu lapor ppn maret aja min kring pajak
Label -> neutral
Prediction -> neutral


### **Voting Classifier (Hard)**

#### Training Model

In [22]:
clf1 = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_df=0.9508799995508697,
        min_df=6,
        ngram_range=(1, 2),
        use_idf=False,
        sublinear_tf=True
    )),
    ('clf', LogisticRegression(
        C=1.304392892275882,
        solver='lbfgs',
        max_iter=685,
        random_state=42
    ))
])

clf2 = ImbPipeline([
        ('tfidf', TfidfVectorizer()),
        ('smote', SMOTE(random_state=42)),
        ('clf', SVC(probability=True))
    ])

clf3 = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', SVC(probability=True))
    ])

In [20]:
voting_clf = VotingClassifier(
    estimators=[
        ('clf1', clf1),
        ('clf2', clf2),
        ('clf3', clf3)
    ],
    voting='hard'
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('clf1',
                              Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(max_df=0.9508799995508697,
                                                               min_df=6,
                                                               ngram_range=(1,
                                                                            2),
                                                               sublinear_tf=True,
                                                               use_idf=False)),
                                              ('clf',
                                               LogisticRegression(C=1.304392892275882,
                                                                  max_iter=685,
                                                                  random_state=42))])),
                             ('clf2',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('smote', SMOTE(random_state=42)),
                                              ('clf', SVC())])),
                             ('clf3',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf', SVC())]))])

#### Evaluation

In [21]:
y_pred = voting_clf.predict(X_test)
print("Accuracy:", classification_report(y_test, y_pred))

Accuracy:               precision    recall  f1-score   support

    negative       0.78      0.89      0.83      1189
     neutral       0.91      0.88      0.90      1723
    positive       0.62      0.17      0.27       162

    accuracy                           0.85      3074
   macro avg       0.77      0.65      0.67      3074
weighted avg       0.85      0.85      0.84      3074



#### Inference

In [39]:
print(f"Text -> {X_test.iloc[148]}")
print(f"Label -> {y_test.iloc[148]}")
print(f"Prediction -> {voting_clf.predict([X_test.iloc[148]])[0]}")

Text -> buyani 123456 hai kak kakak sedang aju mohon surat terang fiskal skf lalu coretax iya sesuai pasal 3 ayat 2 per03p j2019 skf beri wajib pajak penuh tentu salah satu nya sampai surat beritahu
Label -> neutral
Prediction -> neutral


### **Voting Classifier (Soft)**

In [28]:
voting_clf_sft = VotingClassifier(
    estimators=[
        ('clf1', clf1),
        ('clf2', clf2),
        ('clf3', clf3)
    ],
    voting='soft'
)

voting_clf_sft.fit(X_train, y_train)

VotingClassifier(estimators=[('clf1',
                              Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(max_df=0.9508799995508697,
                                                               min_df=6,
                                                               ngram_range=(1,
                                                                            2),
                                                               sublinear_tf=True,
                                                               use_idf=False)),
                                              ('clf',
                                               LogisticRegression(C=1.304392892275882,
                                                                  max_iter=685,
                                                                  random_state=42))])),
                             ('clf2',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('smote', SMOTE(random_state=42)),
                                              ('clf', SVC(probability=True))])),
                             ('clf3',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               SVC(probability=True))]))],
                 voting='soft')

#### Evaluation

In [29]:
y_pred = voting_clf_sft.predict(X_test)
print("Accuracy:", classification_report(y_test, y_pred))

Accuracy:               precision    recall  f1-score   support

    negative       0.79      0.87      0.83      1189
     neutral       0.90      0.89      0.90      1723
    positive       0.58      0.21      0.31       162

    accuracy                           0.85      3074
   macro avg       0.76      0.66      0.68      3074
weighted avg       0.84      0.85      0.84      3074



#### Inference

In [35]:
print(f"Text -> {X_test.iloc[844]}")
print(f"Label -> {y_test.iloc[844]}")
print(f"Prediction -> {voting_clf_sft.predict([X_test.iloc[844]])[0]}")

Text -> obat pusing gw sm coretax url
Label -> negative
Prediction -> negative


## **Word2Vec**

In [27]:
class Word2VecVectorizer(BaseEstimator, TransformerMixin):
      def __init__(self, vector_size=100, min_count=1, window=5, epochs=10):
          self.vector_size = vector_size
          self.min_count = min_count
          self.window = window
          self.epochs = epochs

      def fit(self, X, y=None):
          self.w2v_model = Word2Vec(
                sentences=X,
                vector_size=self.vector_size,
                min_count=self.min_count,
                window=self.window,
                epochs=self.epochs
            )
          return self

      def transform(self, X):
          return np.array([
                np.mean(
                    [self.w2v_model.wv[word] for word in words if word in self.w2v_model.wv]
                    or [np.zeros(self.vector_size)], axis=0)
                for words in X
            ])

In [28]:
X_train_tok = X_train.apply(lambda x: word_tokenize(x))
X_test_tok = X_test.apply(lambda x: word_tokenize(x))

### **Logistic Regression**

#### Hyperparameter Tuning

In [29]:
def objective(trial):
    w2v_params = {
        'vector_size': trial.suggest_int('vector_size', 100, 300),
        'min_count': trial.suggest_int('min_count', 1, 3),
        'window': trial.suggest_int('window', 2, 10),
        'epochs': trial.suggest_int('epochs', 5, 20),
    }

    lr_params = {
        "C": trial.suggest_float("C", 1e-4, 100, log=True),
        "solver": trial.suggest_categorical("solver", ["liblinear", "lbfgs"]),
        "max_iter": trial.suggest_int("max_iter", 200, 1000),
        "random_state": 42,
    }

    pipeline = Pipeline([
        ('w2v', Word2VecVectorizer(**w2v_params)),
        ('clf', LogisticRegression(**lr_params))
    ])

    pipeline.fit(X_train_tok, y_train)
    y_pred = pipeline.predict(X_test_tok)
    acc = accuracy_score(y_test, y_pred)
    return acc

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2025-05-06 13:11:00,452] A new study created in memory with name: no-name-4b5ef1ab-64a1-4d54-9f2e-e6b0ba3ddc4e
[I 2025-05-06 13:11:17,075] Trial 0 finished with value: 0.8253090435914119 and parameters: {'vector_size': 280, 'min_count': 2, 'window': 8, 'epochs': 20, 'C': 0.0045777131219436745, 'solver': 'lbfgs', 'max_iter': 320}. Best is trial 0 with value: 0.8253090435914119.
[I 2025-05-06 13:11:28,787] Trial 1 finished with value: 0.8259596616785947 and parameters: {'vector_size': 267, 'min_count': 2, 'window': 2, 'epochs': 19, 'C': 0.23949940972878728, 'solver': 'lbfgs', 'max_iter': 838}. Best is trial 1 with value: 0.8259596616785947.
[I 2025-05-06 13:11:35,592] Trial 2 finished with value: 0.8223812621990891 and parameters: {'vector_size': 131, 'min_count': 2, 'window': 3, 'epochs': 13, 'C': 0.016628163376528428, 'solver': 'lbfgs', 'max_iter': 295}. Best is trial 1 with value: 0.8259596616785947.
[I 2025-05-06 13:11:42,093] Trial 3 finished with value: 0.8217306441119063 and pa

#### Train Model

Sebelum melakukan hyperparameter tuning, model sudah ditrain dengan parameter default yang mendapatkan akurasi 0.82

In [30]:
best_params = study.best_params

logistic_w2v = Pipeline([
    ('w2v', Word2VecVectorizer(
        vector_size=best_params['vector_size'],
        min_count=best_params['min_count'],
        window=best_params['window'],
        epochs=best_params['epochs']
    )),
    ('clf', LogisticRegression(
        C=best_params['C'],
        solver=best_params['solver'],
        max_iter=best_params['max_iter'],
        random_state=42
    ))
])

logistic_w2v.fit(X_train_tok, y_train)

Pipeline(steps=[('w2v',
                 Word2VecVectorizer(epochs=18, vector_size=226, window=6)),
                ('clf',
                 LogisticRegression(C=78.5338767653747, max_iter=454,
                                    random_state=42, solver='liblinear'))])

#### Evaluation

In [31]:
y_pred = logistic_w2v.predict(X_test_tok)

print("Accuracy:", classification_report(y_test, y_pred))

Accuracy:               precision    recall  f1-score   support

    negative       0.77      0.85      0.81      1189
     neutral       0.89      0.88      0.88      1723
    positive       0.51      0.20      0.29       162

    accuracy                           0.83      3074
   macro avg       0.72      0.64      0.66      3074
weighted avg       0.82      0.83      0.82      3074



#### Inference

In [35]:
print(f"Text -> {X_test.iloc[694]}")
print(f"Label -> {y_test.iloc[694]}")
print(f"Prediction -> {logistic_w2v.predict([X_test_tok.iloc[694]])[0]}")

Text -> kring pajak buat suket pp 55 d coretax coba buat d layan wp buat mohon surat nya simp tgl 7 feb smpai skrg status nya masi mohon blm d sampaikan lanjut nya bagaimana
Label -> neutral
Prediction -> neutral
